## Keras Functional API: Building Non-Linear Neural Networks

### 1. Introduction and Context

The Keras Functional API is a powerful tool for continuing deep learning studies, enabling the creation of complex neural network topologies.

*   **Previous Approach:** All models built previously (e.g., ANNs or CNNs) utilized the **Sequential API**.
*   **Sequential API Limitation:** Sequential models follow a **linear topology**, meaning layers are arranged one after the other. They typically handle a single input and produce a single output.
*   **Functional API Purpose:** To create **non-linear neural networks**. This includes handling **Multi-Input**, **Multi-Output**, and utilizing **shared layers**.

Once you master the Functional API, your "imagination is the limit" for creating any neural network topology.

### 2. Why Non-Linear Topologies are Necessary

Complex problems often require branching or merging paths that linear topologies cannot handle.

| Problem Example | Input(s) | Output(s) | Topology Requirements |
| :--- | :--- | :--- | :--- |
| **Face Analysis** | Image of a human face | Age (Regression) & Emotion (Classification) | Input features pass through a shared CNN base, then **branch** into two separate fully connected pathways. |
| **E-commerce Price Prediction** | Product Metadata (tabular), Product Description (text), Product Image (image) | Price Prediction | Requires three separate neural networks (e.g., RNN for text, CNN for image). Features extracted from all three must be **concatenated** (joined) before final prediction. |

In both cases, the **Sequential API fails** because it cannot handle these non-linear architectures.

### 3. Basic Functional Model Construction

This example shows how to predict two things (Age and Place) from a single tabular input (Yearly Salary, Height, Marital Status).

#### A. Architecture (Multi-Output)

1.  Input Layer (3 columns).
2.  Dense Layer (e.g., 64 neurons, ReLU).
3.  **Branching Point**.
    *   Branch 1: Output 1 (Age prediction; e.g., 1 neuron, Linear activation for regression).
    *   Branch 2: Output 2 (Place/City prediction; e.g., 2 neurons, implied Softmax/Sigmoid for classification).

#### B. Implementation Steps

1.  **Import Necessary Classes:**
    ```python
    from keras.models import Model
    from keras.layers import Input, Dense  # And other required layers
    ```
   

2.  **Define the Input Layer(s):** Use the `Input` class and specify the expected shape. You must give every layer a name.
    ```python
    input_layer = Input(shape=3, name='Input')
    ```
   

3.  **Define and Connect Hidden Layers:** When defining a layer, pass the previous layer's output to it.
    ```python
    hidden_one = Dense(64, activation='relu', name='Hidden_One')(input_layer)
    # The output of the layer is the connection point (X)
    ```
   

4.  **Define Output Branches (Branching):**
    ```python
    output_age = Dense(1, activation='linear', name='Age_Output')(hidden_one)  # Connected to hidden_one
    output_place = Dense(2, activation='softmax', name='Place_Output')(hidden_one) # Connected to hidden_one
    ```
   

5.  **Instantiate the Model:** Define the `inputs` and `outputs`. Multiple inputs or outputs must be passed as a list.
    ```python
    model = Model(inputs=input_layer, outputs=[output_age, output_place])
    ```
   

### 4. Complex Architecture: Multi-Input and Concatenation

This approach is required when different data types (e.g., tabular and image data) must be processed by separate networks and then combined.

*   **Architecture Components:** Two separate branches (e.g., Branch A processing Input A, and Branch B processing Input B).
*   **Feature Extraction:** Each branch extracts features using its own set of layers.
*   **Concatenation:** The final feature outputs from both branches (e.g., `X1` and `Y2`) are combined using a `Concatenate` layer.
    ```python
    combined = concatenate([X1, Y2])
    ```
   
*   **Final Layers:** The combined feature layer (`combined`) is then passed through subsequent Dense layers to reach the single final output.

### 5. Case Study: Transfer Learning with Multi-Output (Age and Gender)

This example utilizes image data (UTK Face dataset) to predict both Age (regression) and Gender (classification) using VGG16.

#### A. Strategy and Architecture

The strategy involves using **Transfer Learning**:

1.  Import the **VGG16 convolutional base** (`include_top=False`).
2.  Pass the VGG16 output through a **Flatten Layer**.
3.  Branch into two separate fully connected layers (one for Age, one for Gender).

#### B. Data Handling

When using a data generator (e.g., `flow_from_dataframe`) for a multi-output problem:

*   The `y` information (labels) must be passed as a **list** containing all output columns (e.g., `['age_column', 'gender_column']`).
*   The generator's `class_mode` parameter must be set to **'multi_output'**.

#### C. Model Compilation

When compiling the functional model, you must specify loss functions and metrics for *each* output.

*   **Losses:** Pass the losses as a list or a dictionary. The losses must correspond to the type of prediction (e.g., specific loss for Age regression, another for Gender classification).
*   **Loss Weights:** You can use a dictionary to assign different weights to the outputs during training (e.g., giving the Gender prediction higher importance/weight).

```python
model.compile(optimizer='adam',
              loss={'Age_Output': 'mse', 'Gender_Output': 'binary_crossentropy'},
              loss_weights={'Age_Output': 0.1, 'Gender_Output': 1.0})
```


The training is completed using the standard `model.fit()` call.

### inception net and resnet are built using functional api in previous notbooks